<a href="https://colab.research.google.com/github/jatinpanjavani/Master/blob/main/Fake_Job_Posting_NLP_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from tensorflow.keras import models, layers, optimizers
import tensorflow

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB


from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

In [15]:
data=pd.read_csv("fake_job_postings.csv")

In [16]:
data.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [17]:
data.shape

(17880, 18)

In [18]:
data=data[['description','fraudulent']]

In [19]:
data=data.dropna()

In [20]:
# Remove Stopwords

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')

data['description']=data['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
data.shape

(17879, 2)

In [22]:
import string

def punctuation_removal(text):
    all_list=[char for char in text if char not in string.punctuation]
    clean_str=''.join(all_list)
    return clean_str

data['description']=data['description'].apply(punctuation_removal)

In [23]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]','',text)
    text = re.sub('\w*\d\w*', '', text)
    return text


In [24]:
data['description']=data['description'].apply(clean_text)

In [25]:
data.head()

,description,fraudulent
0,fastgrowing james beard awardwinning online f...,0
1,organised focused vibrant awesomedo passion...,0
2,our client located houston actively seeking ex...,0
3,the company esri environmental systems resear...,0
4,job title itemization review managerlocation f...,0


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv=TfidfVectorizer(max_features=5000)

x=cv.fit_transform(data['description']).toarray()
y=data['fraudulent']

In [27]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [28]:
rm=RandomForestClassifier()

In [29]:
rm.fit(x_train,y_train)

RandomForestClassifier()

In [30]:
rm_pred=rm.predict(x_test)

In [31]:
accuracy_score(y_test,rm_pred)

0.9781879194630873

In [32]:
def build_model():
    sequences = layers.Input(shape=(5000,))
    embedded = layers.Embedding(5000, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
model = build_model()

In [33]:
model.fit(
    x_train, 
    y_train, 
    batch_size=128,
    epochs=2,
    validation_data=(x_test, y_test), )

Epoch 1/2
112/112 [==============================] - 369s 3s/step - loss: 0.2422 - binary_accuracy: 0.9435 - val_loss: 0.2311 - val_binary_accuracy: 0.9491
Epoch 2/2
112/112 [==============================] - 363s 3s/step - loss: 0.1938 - binary_accuracy: 0.9522 - val_loss: 0.2073 - val_binary_accuracy: 0.9491


In [34]:
preds = model.predict(x_test)
print('Accuracy score: {:0.4}'.format(accuracy_score(y_test, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(y_test, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(y_test, preds)))

Accuracy score: 0.9491
F1 score: 0.0
ROC AUC score: 0.5001


In [41]:
def build_rnn_model():
    sequences = layers.Input(shape=(5000,))
    embedded = layers.Embedding(5000
                                , 64)(sequences)
    x = layers.CuDNNGRU(128, return_sequences=True)(embedded)
    x = layers.CuDNNGRU(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
rnn_model = build_rnn_model()

AttributeError: ignored

In [ ]:
rnn_model.fit(
    x_train, 
    y_train, 
    batch_size=128,
    epochs=10,
    validation_data=(x_test, y_test), )

In [ ]:
preds = rnn_model.predict(x_test)
print('Accuracy score: {:0.4}'.format(accuracy_score(y_test, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(y_test, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(y_test, preds)))

In [42]:
gb=GradientBoostingClassifier()

In [43]:
gs=GaussianNB()

In [44]:
gb.fit(x_train,y_train)

GradientBoostingClassifier()

In [45]:
gb_pred=gb.predict(x_test)

In [46]:
accuracy_score(y_test,gb_pred)

0.9697986577181208

In [47]:
gs.fit(x_train,y_train)

GaussianNB()

In [48]:
gs_pred=gs.predict(x_test)

In [49]:
accuracy_score(y_test,gs_pred)

0.9133109619686801